In [1]:
import os
from langchain.prompts import PromptTemplate
# from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Prompt

In [2]:
prompt_template = """
    You are an assistant that provides useful information about Bidhan. Use the following pieces of retrieved context to answer the question. 
    If the question is about Bidhan's education, summarize his degrees, institutions, and fields of study.
    If you don't know the answer, just say that you don't know. Keep the answer concise.
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate(template= prompt_template)
PROMPT

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant that provides useful information about Bidhan. Use the following pieces of retrieved context to answer the question. \n    If the question is about Bidhan's education, summarize his degrees, institutions, and fields of study.\n    If you don't know the answer, just say that you don't know. Keep the answer concise.\n    {context}\n    Question: {question}\n    Answer:")

# Model

In [3]:
llm = OllamaLLM(model="llama3.2")
llm

OllamaLLM(model='llama3.2')

In [4]:
# testing the llm
llm.invoke("Hello!")

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

# Document Loaders
Data for the context was sourced from my resume and my personal information and beliefs which were all put into two files available in the `data` folder.

In [5]:
# loading documents
all_documents = []

files = ['./data/aboutme.txt', './data/cv.txt']
for file in files:
    loader = TextLoader(file)
    documents = loader.load()
    all_documents.extend(documents)

In [6]:
all_documents

[Document(metadata={'source': './data/aboutme.txt'}, page_content='My Personal Information:\n\nName: Bidhan Bajracharya\nAge: 23 years (as of 2025)\nBirthdate: September 14, 2001\nGender: Male\nNationality: Nepalese\nHeight: 5\'10"\nSkin tone: Brown\n\nEducation Background:\n- Master’s Degree: Data Science and Artificial Intelligence (DSAI)\n    - Institution: Asian Institute of Technology (AIT)\n\n- Bachelor’s Degree: BSc (Hons) in Computing\n    - Institution: London Metropolitan University\n\nProfessional Experience:\n- Software Engineer (1 year)\n    - Industry: Education\n- Currently not employed\n\nTechnical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy, Matplotlib, Scikit-Learn\n\nBeliefs & Philosophy:\n- Technology as a Double-Edged Sword:– If used ethically, technology can drive societal progress, but if misused, it can lead to negative consequences.\n- Data-Driven Decision Making:– Makin

## Document Transformers

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc = text_splitter.split_documents(all_documents)

In [8]:
doc 

[Document(metadata={'source': './data/aboutme.txt'}, page_content='My Personal Information:\n\nName: Bidhan Bajracharya\nAge: 23 years (as of 2025)\nBirthdate: September 14, 2001\nGender: Male\nNationality: Nepalese\nHeight: 5\'10"\nSkin tone: Brown\n\nEducation Background:\n- Master’s Degree: Data Science and Artificial Intelligence (DSAI)\n    - Institution: Asian Institute of Technology (AIT)\n\n- Bachelor’s Degree: BSc (Hons) in Computing\n    - Institution: London Metropolitan University\n\nProfessional Experience:\n- Software Engineer (1 year)\n    - Industry: Education\n- Currently not employed\n\nTechnical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy, Matplotlib, Scikit-Learn'),
 Document(metadata={'source': './data/aboutme.txt'}, page_content='Technical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy,

## Text Embedding Models

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\Bidhan\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Vector Stores

In [10]:
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [11]:
db_file_name = 'personal_info'

# load the vector store if it exists
if os.path.exists(os.path.join(vector_path, db_file_name)):
        vectordb = FAISS.load_local(
            folder_path=os.path.join(vector_path, db_file_name),
            embeddings=embedding_model,
            allow_dangerous_deserialization=True # because I'm sure the file is safe
        )
else:
    vectordb = FAISS.from_documents(
        documents = doc,
        embedding = embedding_model
    )

    vectordb.save_local(
        folder_path = os.path.join(vector_path, db_file_name)
    )

## Retrievers

In [12]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [13]:
retriever.get_relevant_documents("What is your name?")

C:\Users\Bidhan\AppData\Local\Temp\ipykernel_19632\3674800939.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is your name?")


[Document(id='8010dec4-47aa-49db-b2e8-24df4382f1da', metadata={'source': './data/cv.txt'}, page_content='- Moneyvest (TypeScript, React.js, AWS Cognito, Ant-Design)\n    - Built an online platform for finance, investing, and trading education.\n    - Implemented AWS Cognito for secure user authentication.\n    - Collaborated with an international client for API optimization and integration.\n    - Designed data visualization graphs to improve user insights.\n\n- Uddum (Vue.js, Nuxt.js)\n    - Developed an online airplane ticket booking platform.\n    - Worked with experienced developers to optimize and resolve issues.\n    - Implemented REST APIs and developed an intuitive UI/UX.\n\nAWARDS AND ACHIEVEMENT:\n- AAA Scholarship Award (2022)\n    - Recognized for exceptional Attitude, Attendance, and Academic performance.\n\nTECHNICAL SKILLS:\nProgramming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\nTools: Git/GitHub, Jupyter Notebook, VS Code, IntelliJ, PyCharm, Docker, Tablea

# Memory

In [14]:
from langchain.memory import ChatMessageHistory

In [15]:
history = ChatMessageHistory()
history

InMemoryChatMessageHistory(messages=[])

In [16]:
# history.add_user_message('hi')
# history.add_ai_message('Hello! I\'m a personal chatbot who knows a lot about Bidhan.Go ahead and ask me anything about him.')
# history.add_user_message('How are you')
# history.add_ai_message('I\'m quite good. How about you?')

In [17]:
history

InMemoryChatMessageHistory(messages=[])

In [18]:
from langchain.memory import ConversationBufferMemory

In [19]:
# memory = ConversationBufferMemory(return_messages = True)
# memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
# memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
# memory.load_memory_variables({})

In [20]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer")
# memory = ConversationBufferMemory(input_key="question", output_key="answer")

C:\Users\Bidhan\AppData\Local\Temp\ipykernel_19632\4142404075.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer")


In [21]:
# memory.save_context({'question':'hi'}, {'answer':'What\'s up?'})
# memory.save_context({"question":'From now on every reply, you can use the context provided. And answer as bidhan himself.'},{'answer': 'Okay I will now answer as Bidhan.'})

# Chain

In [22]:
from langchain.chains import RetrievalQA

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [24]:
result = qa_chain({"query": "What is my birthdate?"})
answer = result.get('result', '')

C:\Users\Bidhan\AppData\Local\Temp\ipykernel_19632\1298083337.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": "What is my birthdate?"})


In [ ]:
sources = []
source_docs = result.get('source_documents', [])
for doc in source_docs:
    source = {
        "content": doc.page_content[:150] + "...",
        "source": doc.metadata.get('source', 'Unknown Source')
    }
    sources.append(source)

In [26]:
sources

[{'content': 'My Personal Information:\n\nName: Bidhan Bajracharya\nAge: 23 years (as of 2025)\nBirthdate: September 14, 2001\nGender: Male\nNationality: Nepalese\nHeight:...',
  'source': './data/aboutme.txt'},
 {'content': '- Moneyvest (TypeScript, React.js, AWS Cognito, Ant-Design)\n    - Built an online platform for finance, investing, and trading education.\n    - Implem...',
  'source': './data/cv.txt'},
 {'content': 'Technical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy, Matpl...',
  'source': './data/aboutme.txt'}]

In [27]:
answer

'Your birthdate is September 14, 2001.'

In [28]:
questions = [
 "How old are you?",
 "What is your highest level of education?",
 "What major or field of study did you pursue during your education?",
 "How many years of work experience do you have?",
 "What type of work or industry have you been involved in?",
 "Can you describe your current role or job responsibilities?",
 "What are your core beliefs regarding the role of technology in shaping society?",
 "How do you think cultural values should influence technological advancements?",
 "As a master's student, what is the most challenging aspect of your studies so far?",
 "What specific research interests or academic goals do you hope to achieve during your time as a master's student?"
]

In [45]:
qa_pairs = []

for idx, question in enumerate(questions, start=1):
    response = qa_chain({"query": question})
    qa_pairs.append({"question": question, "answer": response["result"]})
    print(f"{idx}) {question}")
    print(f"Answer: {response['result']}\n")

1) How old are you?
Answer: You are 23 years old (as of 2025).

2) What is your highest level of education?
Answer: I don't know the answer to this question. The provided context doesn't mention Bidhan's highest level of education or any specific degree held by him.

3) What major or field of study did you pursue during your education?
Answer: During my education, I pursued a Master's degree in Bidhan. Unfortunately, the provided context does not specify the exact major or field of study pursued by Bidhan.

4) How many years of work experience do you have?
Answer: I don't know the exact number of years of work experience Bidhan has. However, he mentioned that his professional experience started in June 2023, but I don't have enough information to determine how long he has been working.

5) What type of work or industry have you been involved in?
Answer: Based on the provided context, I can tell that Bidhan has been involved in developing and implementing various projects related to fin

In [46]:
import json

with open("qa_pairs.json", "w", encoding="utf-8") as f:
    json.dump(qa_pairs, f, ensure_ascii=False, indent=4)

In [48]:
# json response
qa_pairs

[{'question': 'How old are you?',
  'answer': 'You are 23 years old (as of 2025).'},
 {'question': 'What is your highest level of education?',
  'answer': "I don't know the answer to this question. The provided context doesn't mention Bidhan's highest level of education or any specific degree held by him."},
 {'question': 'What major or field of study did you pursue during your education?',
  'answer': "During my education, I pursued a Master's degree in Bidhan. Unfortunately, the provided context does not specify the exact major or field of study pursued by Bidhan."},
 {'question': 'How many years of work experience do you have?',
  'answer': "I don't know the exact number of years of work experience Bidhan has. However, he mentioned that his professional experience started in June 2023, but I don't have enough information to determine how long he has been working."},
 {'question': 'What type of work or industry have you been involved in?',
  'answer': "Based on the provided context, 

# Analysis and Problem Solving

In [30]:
# code here...

In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [32]:
prompt_template = """
    You are an assistant that provides useful information about Bidhan. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    {context}
    Question: {input}
    Answer:
    """.strip()

PROMPT2 = PromptTemplate(template= prompt_template)

In [33]:
document_chain=create_stuff_documents_chain(llm, PROMPT2)

In [34]:
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [35]:
response=retrieval_chain.invoke({"input":"What is your highest level of education?"})

In [36]:
response

{'input': 'What is your highest level of education?',
 'context': [Document(id='9bc22508-5e07-4ab5-9a1f-d41661a5e40a', metadata={'source': './data/aboutme.txt'}, page_content='Technical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy, Matplotlib, Scikit-Learn\n\nBeliefs & Philosophy:\n- Technology as a Double-Edged Sword:– If used ethically, technology can drive societal progress, but if misused, it can lead to negative consequences.\n- Data-Driven Decision Making:– Making informed decisions based on data and logical reasoning is crucial in both personal and professional life.\n- Empathy & Ethical Responsibility:– The impact of technology on people’s lives should always be considered, and ethical practices should be prioritized in development.\n- Cultural Identity in a Globalized World:– While globalization connects us, preserving cultural uniqueness is essential for diversity and innovation.\n- Cre

In [37]:
for idx, question in enumerate(questions, start=1):
    response = retrieval_chain.invoke({"input": question})
    print(f"{idx}) {question}")
    print(f"Answer: {response['answer']}\n")

1) How old are you?
Answer: As of 2025, Bidhan Bajracharya is 23 years old.

2) What is your highest level of education?
Answer: I don't have information on the highest level of education provided by Bidhan. However, I can tell you that they hold an award (AAA Scholarship Award) and are currently pursuing a Master's degree.

3) What major or field of study did you pursue during your education?
Answer: I don't have information on the specific major or field of study pursued by Bidhan. The provided context does not mention this detail.

4) How many years of work experience do you have?
Answer: Bidhan has 1 year of work experience as a Junior Software Engineer. He does not have any prior work experience before that. His professional experience started in June 2023 and ended in May 2024.

5) What type of work or industry have you been involved in?
Answer: I've been involved in finance, investing, and trading education, specifically building an online platform that utilizes technology such 

In [38]:
vectordb.similarity_search("What is Bidhan's major?")

[Document(id='9bc22508-5e07-4ab5-9a1f-d41661a5e40a', metadata={'source': './data/aboutme.txt'}, page_content='Technical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy, Matplotlib, Scikit-Learn\n\nBeliefs & Philosophy:\n- Technology as a Double-Edged Sword:– If used ethically, technology can drive societal progress, but if misused, it can lead to negative consequences.\n- Data-Driven Decision Making:– Making informed decisions based on data and logical reasoning is crucial in both personal and professional life.\n- Empathy & Ethical Responsibility:– The impact of technology on people’s lives should always be considered, and ethical practices should be prioritized in development.\n- Cultural Identity in a Globalized World:– While globalization connects us, preserving cultural uniqueness is essential for diversity and innovation.\n- Creativity & Problem-Solving:– Every challenge is an opportunity for 